In [1]:
import json
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alpha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import re
#import sentencepiece as spm
import os
import json

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s.?!]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '\s+'         # tab to whitespace
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '\d\.\d+'         
    text = re.sub(pattern=pattern, repl='USRNUM', string=text)
    pattern = '\d+\.'         
    text = re.sub(pattern=pattern, repl='USRSEQ', string=text)
    pattern = '\d+'         
    text = re.sub(pattern=pattern, repl='USRNUM', string=text)
    return text  

# 4 Multi-classification model data 생성

In [3]:
file_names = []
for root, dirs, files in os.walk('data/petitions/2017-and-2019'):
    for fname in files:
        full_fname = os.path.join(root, fname)

        file_names.append(full_fname)

In [4]:
c_list = []
for i, file_name in enumerate(file_names):
    with open(file_name, "r", encoding='utf-8', errors='ignore') as in_file:
        datas = in_file.readlines()

        for j in range(len(datas)):
            data = json.loads(datas[j])
            content = clean_str(data['content'])
            category = data['category']
            begin = data['begin']
            c_list.append((content, category, begin))
            
    print("{0} file has been converted".format(i+1))
print("Total {0} datas has been converted".format(len(c_list)))

1 file has been converted
2 file has been converted
3 file has been converted
4 file has been converted
5 file has been converted
6 file has been converted
7 file has been converted
8 file has been converted
9 file has been converted
10 file has been converted
11 file has been converted
12 file has been converted
13 file has been converted
Total 138201 datas has been converted


In [5]:
import pandas as pd
from sklearn.utils import shuffle

In [6]:
df = pd.DataFrame(c_list, columns=['contents','category','begin']) 

In [7]:
df.head(5)

,contents,category,begin
0,안녕하세요. 현재 사대 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어...,육아/교육,2017-08-25
1,서울시에서는 USRNUM년부터 정수지와 배수지 청소용 화학제품인 세정제에 대하여 첨...,안전/환경,2017-08-25
2,얼마 전 살충제 계란 파동 등으로 씨끄러웠던 중 친환경업체한살림 등의 DDT파동으로...,기타,2017-08-25
3,길고양이들 밥주고있는 사람입니다. 최근에 동네주민과 트러블이 생겨 싸움이 일어났습니...,반려동물,2017-08-25
4,안녕하세요 지인분이 성관계 몰래카메라가 유출되어 방송통신 심의 위원회에 접수해서 처...,인권/성평등,2017-08-25


In [8]:
len(df)

138201

In [9]:
df.groupby('category')['category'].count()

category
경제민주화           3040
교통/건축/국토        6783
기타             18332
농산어촌             677
문화/예술/체육/언론     4626
미래              6115
반려동물            1461
보건복지            9015
성장동력            1482
안전/환경          13320
외교/통일/국방        6926
육아/교육          10260
인권/성평등         14923
일자리             6339
저출산/고령화대책       1354
정치개혁           26951
행정              6597
Name: category, dtype: int64

In [10]:
# 기타 항목 제거
df = df.loc[df['category'] != '기타']

In [11]:
len(df)

119869

In [12]:
'''category_dictionary = {
    '경제민주화' : '0', 
    '교통/건축/국토' : '1', 
    '농산어촌' : '2',
    '문화/예술/체육/언론' : '3',
    '미래' : '4',
    '반려동물' : '5',
    '보건복지' : '6',
    '성장동력' : '7',
    '안전/환경' : '8',
    '외교/통일/국방' : '9',
    '육아/교육' : '10',
    '인권/성평등' : '11',
    '일자리' : '12',
    '저출산/고령화대책' : '13',
    '정치개혁' : '14',
    '행정' : '15',
    '기타' : '16',
} '''

category_dictionary = {
    '경제민주화' : '0', 
    '교통/건축/국토' : '1', 
    '농산어촌' : '2',
    '문화/예술/체육/언론' : '3',
    '미래' : '4',
    '반려동물' : '5',
    '보건복지' : '6',
    '성장동력' : '7',
    '안전/환경' : '8',
    '외교/통일/국방' : '9',
    '육아/교육' : '10',
    '인권/성평등' : '11',
    '일자리' : '12',
    '저출산/고령화대책' : '13',
    '정치개혁' : '14',
    '행정' : '15',
} 

In [13]:
df['category_label'] = df['category'].map(category_dictionary)

In [14]:
df.head(5)

,contents,category,begin,category_label
0,안녕하세요. 현재 사대 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어...,육아/교육,2017-08-25,10
1,서울시에서는 USRNUM년부터 정수지와 배수지 청소용 화학제품인 세정제에 대하여 첨...,안전/환경,2017-08-25,8
3,길고양이들 밥주고있는 사람입니다. 최근에 동네주민과 트러블이 생겨 싸움이 일어났습니...,반려동물,2017-08-25,5
4,안녕하세요 지인분이 성관계 몰래카메라가 유출되어 방송통신 심의 위원회에 접수해서 처...,인권/성평등,2017-08-25,11
5,대통령님 민주주의 사회에서는 국공립과 사립은 공존하여 서로 경쟁 발전해야하며 시급한...,육아/교육,2017-08-25,10


In [15]:
df["contents_length"]= df["contents"].str.len() 

In [16]:
# contens 갯수 확인 후 제거
df = df.loc[(df['contents_length'] > 20) & (df['contents_length'] < 30000)] 

In [17]:
df.sort_values(by=['contents_length'])

,contents,category,begin,category_label,contents_length
37249,이명박 전대통령 출국을금지 했으면합니다,정치개혁,2017-11-11,14,21
114023,사법개혁??? 가능 하긴 한겁니꽈???,정치개혁,2019-03-04,14,21
37941,출국금지시키고 검찰수사 받게 해야합니다,정치개혁,2017-11-11,14,21
106987,진심 이따구로 대답할꺼면 왜 만들었는지,인권/성평등,2019-02-22,11,21
28230,특별법제정해서라도 사회에서격리시켜주세요,안전/환경,2017-11-09,8,21
...,...,...,...,...,...
73013,모든 국민이 피해자인 아파트비리에 대한 적폐청산을 해 주세요 문재인 대통령님! 안녕...,미래,2019-01-03,4,25937
73197,대한민국을 휩쓸고 있는 예멘난민문제..누구보다 앞장서서 해결해야 하는 전담부서인 여...,정치개혁,2019-01-03,14,27279
96860,이해식 대변인 브리핑 평창올림픽 USRNUM년으로 돌아보는 격세지감의 한반도 평화 ...,성장동력,2019-02-09,7,27399
124190,대한민국을 휩쓸고 있는 예멘난민문제..누구보다 앞장서서 해결해야 하는 전담부서인 여...,인권/성평등,2019-03-14,11,27403


In [18]:
df = shuffle(df)

In [19]:
df.head(5)

,contents,category,begin,category_label,contents_length
51637,특활비 과연 제대로 이용 되고 있나요? 국회의원 에게 지급되는 비용에 대한 투명성조...,정치개혁,2017-11-28,14,320
84741,대한민국의 광화문 광장은 현대사의 상징입니다. 보수하고 고쳐나가고 더 좋은 환경으로...,교통/건축/국토,2019-01-21,1,259
10248,휠체어를 사용하는 지제장애 USRNUM급입니다. 장애인은 대중교통 이용이 너무 어려...,교통/건축/국토,2017-09-06,1,711
545,엄연한 회원들을 팔아서 유지하고 있는 대한민국USRNUM전몰군경 유족회와 유자녀회?...,행정,2017-08-31,15,599
91640,김경수 경남도지사 드로킹 판결을 위해 법정에선 허익범 특검이 드로킹 김동원 씨 진술...,정치개혁,2019-01-30,14,204


In [20]:
df.groupby('category')['category'].count()

category
경제민주화           2983
교통/건축/국토        6646
농산어촌             653
문화/예술/체육/언론     4473
미래              5562
반려동물            1415
보건복지            8503
성장동력            1396
안전/환경          12284
외교/통일/국방        6673
육아/교육           9448
인권/성평등         13280
일자리             6174
저출산/고령화대책       1325
정치개혁           20571
행정              6380
Name: category, dtype: int64

In [31]:
file_path = "data/petitions/petition.mcc.cls-2-7-13.train"
df_by_cls = df.loc[(df['category_label'] == '2') | (df['category_label'] == '7') | (df['category_label'] == '13')]
df_by_cls.to_csv(file_path, columns=['contents','category_label'],sep='\t', encoding='utf-8')

In [34]:
df_cls = pd.read_csv(file_path, names=['idx', 'contents','category_label'],sep='\t', header=0)
df_2018 = pd.read_csv('data/petitions/petition.mcc.without-etc.2018.clean1.train', names=['idx', 'contents','category_label'],sep='\t', header=0)

In [42]:
df_cls.head()

,idx,contents,category_label
0,16279,USRNUM대 초반입니다. 난임으로 인공수정 체외수정까지 기존 지원 다썼습니다. U...,13
1,17098,정읍도 교육시설이 희박해 대학가기 힘듭니다.,2
2,17830,각나라의 최첨단 기술을 보여주는 올림픽 무대에서 멋진 기술로 USRNUM차산업혁명의...,7
3,52141,문제가 있는 부분은 개선하면 됩니다. 개선도 천천히 폐부를 잘라내면 됩니다. 확대해...,13
4,73018,신규간호사 USRNUM만명 양성 의사도 신규로 USRNUM만명 양성해야 정부가 간호...,7


In [41]:
df_2018.head()

,idx,contents,category_label
0,60830,직접 진상을 조사해야지 무슨자체조사? 박영선 의원은 출입카드 없이 들어간점 직접 해...,3
1,238910,A형B형독감을예방하려면USRNUM가를맞아야하는데USRNUM만운너무비싸요 백신무료로맞...,10
2,94782,저는 USRNUM년생이자 롯데구단 팬인 USRNUM살 남자 청년입니다... 제가 구...,3
3,98040,경고 부분 발췌는 오용될 수 있는 내용을 포함하고 있으므로 전문이 아닌 부분 발췌와...,14
4,76842,물론 그들이 노력해서 얻은 것이겠지만 나쁜짓 안했어도 보통 월급 쟁이면 최저시급만 ...,14


In [46]:
df_concat = pd.concat([df_cls, df_2018])

In [47]:
df_concat = shuffle(df_concat)

In [48]:
df_concat.head()

,idx,contents,category_label
36788,228700,어느당이라고는 말하지않겠습니다 민주정권이라고 너무함부로대하고 불법을 맘것저지르는일은...,15
66098,132591,박근혜정부때 서민들의 주머니을 생각을 안하고털기위해 USRNUM원이라는 금액을 올린...,14
47240,19763,저희는 양주시 장흥에서 화훼업을 하고있는 농가입니다. 일률적으로 고르게 작물을 재배...,2
2839,70019,투자는 개인이 번 돈으로 개인의 의지로 하고 있습니다. 나라에서 보호 규제 등 전혀...,7
52538,163471,작금의 대한민국의 국가인권위원회는 누구를 위한 인권위원회 인가요? 강럭범죄를 저지른...,11


In [51]:
df_concat.groupby('category_label')['category_label'].count()

category_label
0      5602
1      9078
2      1203
3      5801
4      5336
5      1145
6      7016
7      3785
8      8528
9      8410
10     6644
11     9205
12     7182
13     2373
14    16217
15     5849
Name: category_label, dtype: int64

In [49]:
attached_file_path = "data/petitions/petition.mcc.attached.train"
df_concat.to_csv(attached_file_path, columns=['contents','category_label'],sep='\t', encoding='utf-8')